In [211]:
import pandas as pd
import numpy as np 
import os

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import NearestNeighbors

In [212]:
dataset = pd.read_csv(os.getcwd() + '/spotify_songs.csv')
dataset.head()

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,11,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052


In [213]:
print(dataset.columns)

Index(['track_id', 'track_name', 'track_artist', 'track_popularity',
       'track_album_id', 'track_album_name', 'track_album_release_date',
       'playlist_name', 'playlist_id', 'playlist_genre', 'playlist_subgenre',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms'],
      dtype='object')


In [214]:
categorical_features = ['track_artist',  
                        'track_album_name', 
                        'playlist_name', 
                        'playlist_genre', 
                        'playlist_subgenre']

numeric_features = ['track_popularity',
                    'danceability', 
                    'energy', 
                    'key', 
                    'loudness', 
                    'mode', 
                    'speechiness',
                    'acousticness', 
                    'instrumentalness', 
                    'liveness', 
                    'valence', 
                    'tempo']

test_categorical = [
                    'playlist_genre',
                    'playlist_subgenre']

In [215]:
from sklearn.preprocessing import LabelEncoder
# LabelEncoder implementation for high-dimensional categorical variables.
# Fully captures dataset, but reuslts in very high inertia and will require many more clusters of KMeans.

label_encoded_columns = [
    'track_artist',
    'track_name',
    'track_album_name'
]

encoded_labels = dict()

for column in label_encoded_columns:
    encoded_labels[column] = LabelEncoder().fit_transform(dataset[column])
encoded_labels_df = pd.DataFrame(encoded_labels)

In [216]:
# Category Encoders, can create link between columns in dataset to reduce dimensionality of artists.
# Not certain what column to link together.

# import category_encoders as ce

# category_encoder = ce.TargetEncoder(cols=['track_artist'])
# artist_encoded = category_encoder.fit_transform(dataset['target_artist'])


# PCA Implementation for track_artist.
# Efficient, but struggling to capture whole dataset of artists.
# Also much harder to explain, as reducing from 10k+ artists.
from sklearn.decomposition import PCA

encoder = OneHotEncoder(sparse_output=False)
artist_encoded = encoder.fit_transform(dataset[['track_artist']])

pca = PCA(n_components=100)
artist_reduced = pca.fit_transform(artist_encoded)

# Want high Cumulative Explained Variance. If this is low, then we do not have enough components to capture the dataset.
print(f"Cumulative explained variance: {sum(pca.explained_variance_ratio_)}")

artist_reduced_df = pd.DataFrame(artist_reduced, columns=[f'PCA_{i+1}' for i in range(artist_reduced.shape[1])])
artist_reduced_df.head()

Cumulative explained variance: 0.1581678008755016


,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,PCA_9,PCA_10,...,PCA_91,PCA_92,PCA_93,PCA_94,PCA_95,PCA_96,PCA_97,PCA_98,PCA_99,PCA_100
0,-0.007359,-0.007090,-0.007279,-0.007324,-0.006998,-0.008953,-0.008361,-0.010410,-0.009665,-0.013765,...,0.000562,0.000453,0.000065,0.000946,0.000491,0.000148,0.000811,-0.000398,-4.194457e-06,-0.000481
1,-0.007523,-0.007310,-0.007561,-0.007702,-0.007453,-0.009576,-0.009129,-0.011464,-0.011170,-0.016097,...,0.000533,0.000452,0.000077,0.000908,0.000456,0.000146,0.000771,-0.000378,-2.579881e-08,-0.000450
2,-0.005079,-0.004392,-0.004109,-0.003652,-0.003144,-0.003877,-0.003089,-0.003654,-0.002688,-0.003690,...,0.068646,-0.096494,0.004360,-0.031825,0.149669,0.132203,0.025790,0.063813,-7.230230e-02,-0.074074
3,-0.018005,-0.037657,0.995126,0.022412,0.010801,0.011815,0.006589,0.007032,0.003668,0.004727,...,0.000238,0.000187,0.000035,0.000384,0.000204,0.000067,0.000342,-0.000166,-5.095453e-06,-0.000195
4,-0.004824,-0.004118,-0.003840,-0.003360,-0.002843,-0.003493,-0.002777,-0.003243,-0.002297,-0.003068,...,0.000371,0.000112,-0.010001,-0.005425,0.003375,-0.021325,-0.004860,-0.005126,-1.867580e-03,0.039192


In [217]:
encoder = OneHotEncoder()
# Changed to use test_categorical, a set of variables that doesn't have the 10k+ entry problem mainly for testing
encoded_categorical = encoder.fit_transform(dataset[test_categorical]).toarray()
encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(test_categorical))

encoded_categorical_df.head()

,playlist_genre_edm,playlist_genre_latin,playlist_genre_pop,playlist_genre_r&b,playlist_genre_rap,playlist_genre_rock,playlist_subgenre_album rock,playlist_subgenre_big room,playlist_subgenre_classic rock,playlist_subgenre_dance pop,...,playlist_subgenre_new jack swing,playlist_subgenre_permanent wave,playlist_subgenre_pop edm,playlist_subgenre_post-teen pop,playlist_subgenre_progressive electro house,playlist_subgenre_reggaeton,playlist_subgenre_southern hip hop,playlist_subgenre_trap,playlist_subgenre_tropical,playlist_subgenre_urban contemporary
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [218]:
scaler = StandardScaler()
scaled_numerical = scaler.fit_transform(dataset[numeric_features])
scaled_numerical_df = pd.DataFrame(scaled_numerical, columns=numeric_features)

scaled_numerical_df.head()

,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.941531,0.642049,1.201614,0.173200,1.367123,0.876177,-0.481362,-0.333898,-0.377953,-0.809230,0.031908,0.042927
1,0.981557,0.490412,0.643317,1.557627,0.585766,0.876177,-0.688642,-0.468670,-0.359177,1.081061,0.782522,-0.777198
2,1.101635,0.138889,1.284529,-1.211227,1.100090,-1.141322,-0.324422,-0.436799,-0.377849,-0.519562,0.439384,0.116227
3,0.701374,0.435271,1.279002,0.450085,0.984309,0.876177,-0.050024,-0.667642,-0.377911,0.089582,-1.001795,0.039953
4,1.061609,-0.033426,0.742815,-1.211227,0.685151,0.876177,-0.702460,-0.432701,-0.377953,-0.692585,0.919777,0.115037


In [219]:
processed_features_df = pd.concat([encoded_labels_df, scaled_numerical_df, encoded_categorical_df], axis=1)
processed_features_df.head()

,track_artist,track_name,track_album_name,track_popularity,danceability,energy,key,loudness,mode,speechiness,...,playlist_subgenre_new jack swing,playlist_subgenre_permanent wave,playlist_subgenre_pop edm,playlist_subgenre_post-teen pop,playlist_subgenre_progressive electro house,playlist_subgenre_reggaeton,playlist_subgenre_southern hip hop,playlist_subgenre_trap,playlist_subgenre_tropical,playlist_subgenre_urban contemporary
0,2782,8898,7614,0.941531,0.642049,1.201614,0.173200,1.367123,0.876177,-0.481362,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6084,12520,10410,0.981557,0.490412,0.643317,1.557627,0.585766,0.876177,-0.688642,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10416,924,985,1.101635,0.138889,1.284529,-1.211227,1.100090,-1.141322,-0.324422,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9215,3020,2798,0.701374,0.435271,1.279002,0.450085,0.984309,0.876177,-0.050024,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5402,17910,14843,1.061609,-0.033426,0.742815,-1.211227,0.685151,0.876177,-0.702460,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [224]:
# from sklearn.pipeline import make_pipeline
# from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

dataset['track_name'] = dataset['track_name'].fillna("Unkown")

X, y = processed_features_df, dataset['track_name']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.15042630937880633
                                                                                                      precision    recall  f1-score   support

                                                              "I TRIED FOR YEARS... NOBODY LISTENED"       0.00      0.00      0.00         0
                                                                                 "This Is Seagull…."       0.00      0.00      0.00         1
                                                                                             $Dreams       0.00      0.00      0.00         0
                                                  $ave Dat Money (feat. Fetty Wap & Rich Homie Quan)       0.00      0.00      0.00         0
                                                                             (Don't Fear) The Reaper       0.00      0.00      0.00         0
                                                                          (I Just) Died In Your Arms       0.00      0.00      

C:\Users\jpwis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jpwis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jpwis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1469: Undefi